In [ ]:
using DrWatson
@quickactivate "QAOA.jl"

In [ ]:
include("../src/QAOA.jl")

using Statistics, BenchmarkTools

using PyPlot
PyPlot.plt.style.use("paper.mplstyle")
using PyCall
np = pyimport("numpy")
nx = pyimport("networkx")
pytime = pyimport("time");

## Scan graph size

### Python

In [ ]:
py"""
import networkx as nx
import pennylane as qml
from pennylane import qaoa
from pennylane import numpy as np
import time

def scan_graph_size(N, p, steps=128):
    graph = nx.gnp_random_graph(N, 0.5, seed=137)
    cost_h, mixer_h = qaoa.min_vertex_cover(graph, constrained=False)
    
    def qaoa_layer(gamma, alpha):
        qaoa.cost_layer(gamma, cost_h)
        qaoa.mixer_layer(alpha, mixer_h)    
    
    wires = range(N)
    depth = p   
    beta = np.array([0.5*(1-j/p) for j in range(1, p)] + [0.5/(4*p)])
    gamma = np.array([0.5*(j-1/2)/p  for j in range(1, p + 1)])
    
    optimizer = qml.GradientDescentOptimizer()
    #optimizer = qml.AdamOptimizer()
    params = np.array([beta, gamma], requires_grad=True)  
    
    def circuit(params, **kwargs):
        for w in wires:
            qml.Hadamard(wires=w)
        qml.layer(qaoa_layer, depth, params[0], params[1])     
    
    # dev = qml.device("qulacs.simulator", wires=wires)
    dev = qml.device("default.qubit", wires=wires)

    @qml.qnode(dev)
    def cost_function(params):
        circuit(params)
        return qml.expval(cost_h)

    start = time.time()
    for i in range(steps):
        params = optimizer.step(cost_function, params)
    end = time.time()

    return end - start

p = 3

nrange = [10] # [5, 10, 15, 20]
data = list(map(lambda n: np.mean([scan_graph_size(n, 2) for _ in range(1)]), nrange))

np.save("../data/test.npy", data)
"""

### Julia

In [ ]:
function scan_graph_size(N, p)
    graph = nx.gnp_random_graph(N, 0.5, seed=137) 
    
    h = -ones(N)
    J = zeros(N, N)
    for edge in graph.edges
        h[edge[1] + 1] += 3/4.
        h[edge[2] + 1] += 3/4.
        J[(edge .+ (1, 1))...] = 3/4.
    end    
    
    problem = QAOA.Problem(p, h, J)
    beta = vcat([0.5(1-j/p) for j in 1:p-1], [0.5/(4p)])
    gamma = [0.5(j-1/2)/p  for j in 1:p]  
#     @belapsed QAOA.gradient_optimize(problem, vcat(beta, gamma))
#     @time QAOA.gradient_optimize(problem, vcat(beta, gamma))
    t_0 = pytime.time()
    cost, params, probabilities = QAOA.optimize_parameters(problem, vcat(beta, gamma))
#     cost, params, probabilities = QAOA.optimize_parameters(problem, vcat(beta, gamma), :LN_COBYLA)
    t_f = pytime.time()
    return t_f - t_0
end

In [ ]:
nrange = [10];
# nrange = [5, 10];
# nrange = [5, 10, 15, 20];

In [ ]:
data = map(n -> scan_graph_size(n, 3), nrange)

In [ ]:
# data = map(n -> mean([scan_graph_size(n, 2) for _ in 1:3]), nrange)
data = map(n -> scan_graph_size(n, 3), nrange)

In [ ]:
np.save("../data/qaoa_yao.npy", data)

In [ ]:
FILE = "../data/qaoa_pennylane.npy"
pennylane_data = np.load(FILE)

FILE = "../data/qaoa_yao.npy"
data = np.load(FILE)

In [ ]:
pennylane_data

In [ ]:
figure(figsize=(4, 3))
semilogy(nrange, data)
semilogy(nrange, pennylane_data)
tight_layout()

In [ ]:
figure(figsize=(4, 3))
plot(nrange, pennylane_data ./ data, "o-")
ylim(1, )
tight_layout()

In [ ]:
FILE = "../data/test.npy"
test_data = np.load(FILE)